In [ ]:
# permutations of actions and queue

In [ ]:
import pandas as pd
import numpy as np
import random
from random import randrange
import time
import os
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
import itertools
from itertools import permutations
from itertools import combinations

from copy import deepcopy

# https://www.redblobgames.com/grids/circle-drawing/

In [ ]:
class world:
    def __init__(self,Size):
        self.Size = Size
        self.Fig = None
        self.Grid = None
        self.Coin_Locations = []
        self.Enemy_Locations = []

    def Generate_Map(world):
        # generate a grid with the dimensions of world.Size
        grid = np.zeros([world.Size,world.Size])
    
        fig = plt.figure(figsize = [5,5])
        plt.xlim(0,world.Size)
        plt.ylim(0,world.Size)

        world.Grid = grid
        world.Fig = fig

    def Add_Enemy(world,Location):
        # add an enemy to the grid at the specified location
        world.Grid[Location[0],Location[1]] = 2
        world.Enemy_Locations.append(Location)
    

In [ ]:
class entity:
    def __init__(self,world):
        self.world = world
        self.IsSpawned = False

        if self.IsSpawned == False:
            self.Location = [0,0]
            
        elif self.IsSpawned == True:
            x_loc = world.Grid[np.where(world.Grid[:,0] == max(world.Grid))]
            print('x: ',x_loc)
            y_loc = world.Grid[np.where(world.Grid[0,:] == max(world.Grid))]
            print('y: ',y_loc)
            self.Location = [x_loc,y_loc]

        self.Speed = 6
        self.Coins = 0


In [ ]:
stage = world(10)
stage.Generate_Map()
actor = entity(stage)

In [ ]:
queue = {
    'action': [],
    'destination': [],
    'metadata': []
}


In [ ]:
# create observer class
class Observer:
    def __init__(self, name):
        self.name = name

    def update(self, queue):
        print(f'{self.name} received: {queue}')


class Observable:
    def __init__(self):
        self.observers = []

    def add_observer(self, observer):
        self.observers.append(observer)

    def remove_observer(self, observer):
        self.observers.remove(observer)

    def notify_observers(self, queue):
        for observer in self.observers:
            observer.update(queue)

    def update_queue(self, queue):
        self.notify_observers(queue)
    

In [ ]:
def adjacent_locations(pos):
    adjacent_locations_list = []
    # for an 8 directional grid
    for i in range(-1,2):
        for j in range(-1,2):
            adjacent_locations_list.append([pos[0]+i,pos[1]+j])
    adjacent_locations_list.remove(pos)
    return adjacent_locations_list

# Chebyshev 
def chebyshev_distance(pos1,pos2):
    return max(abs(pos1[0]-pos2[0]),abs(pos1[1]-pos2[1]))

In [ ]:
subaction_dict = {
    '0': 'Move',                # needs a target
    '1': 'Object (Bonus)',      # needs a target
    '2': 'Attack',              # needs a target
    '3': 'Dash',
    '4': 'Disengage',
    '5': 'Dodge',
    '6': 'Object',              # needs a target
}

In [ ]:
##### Fleshing Out the Move Action #####
# the Move Action moves them 1 Space to a new Space
# the new Space is the target
# and the target choices the entity has to pick from  

# because of the problem of any Move action after the first not knowing where the entity is, which of the two solutions is more efficient:
# 1. Rendering: every time a move action is in the permutation list, render the world and the entity's location
# 2. Location Incorporation: when randomly generating the permutations, the locations within 6 spaces of the entity are included in the permutation list, the validity rules are updated accordingly

def movement_range(entity):
    entity_pos = entity.Location
    movement_range = []
    for x in range(-entity.Speed,entity.Speed+1):
        for y in range(-entity.Speed,entity.Speed+1):
            if abs(x) + abs(y) <= entity.Speed:
                movement_range.append([entity_pos[0]+x,entity_pos[1]+y])
    #print('Spaces within Movement Range: ',len(movement_range))
    return movement_range

# what if there was a function called flesh_out_move_perms that went through the list of initial permutations (that only had 0-6) and where ever there was a 0, 
# it would create a new permutation in which a target was chosen from the movement_range of the entity
# then a second function to check the validity of movement

def validate_movement(perms,entity):

    for perm in perms:
        #print(perm)
        if 0 in perm:
            # moves_taken should be a list of all the items in prem that are a list with 0 as the first element
            moves_taken = [item[1] for item in perm if type(item) == list and item[0] == 0]

            # check that the distance from one item to the next is one, if not, remove the permutation
            for i in range(len(moves_taken)-1):
                # if the first space is not 1 space or less from the entity's location, remove the permutation
                if chebyshev_distance(moves_taken[i],entity.Location) > 1:
                    perms.remove(perm)
                    break
                
                # use chevyshev distance for this
                if chebyshev_distance(moves_taken[i],moves_taken[i+1]) > 1:
                    perms.remove(perm)
                    break


In [ ]:
# What Should Be Happening?


# this example permutation is an action-series representing two move actions
# these permutations can range from 1 to 8 numbers in length, and contain numbers 0 through 6. 
# I'm only trying to flesh out the permutations that contain 0s, and the 0s represent move actions
# Example Permutation: [0,0]

# Step 1
# Create 8 new ones
# [ [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0] ]

# Step 2
# Replace the first 0 in each with an adjacent location
# [ [[0,[-1,-1]],0], [[0,[-1,1]],0], [[0,[1,-1]],0], [[0,[1,0]],0], [[0,[0,1]],0], [[0,[-1,0]],0], [[0,[0,-1]],0], [[0,[1,1]],0] ]

# Step 3
# Create 8 copies of each of the above
# [ [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0], [[0,[-1,-1]],0]]
# [ [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0],  [[0,[-1,1]],0]]
# [ [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0],  [[0,[1,-1]],0]]
# [ [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0],   [[0,[1,0]],0]]
# [ [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0],   [[0,[0,1]],0]]
# [ [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0],  [[0,[-1,0]],0]]
# [ [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0],  [[0,[0,-1]],0]]
# [ [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0],   [[0,[1,1]],0]]

# Step 4
# Replace the second 0 in each with an adjacent location to the first 0
# [ [ [0,[-1,-1]],[0,[-2,-2]] ], [ [0,[-1,-1]],[0,[-1,-2]] ], [ [0,[-1,-1]],[0,[0,-2]] ], [ [0,[-1,-1]],[0,[0,-1]] ], [ [0,[-1,-1]],[0,[0,0]] ], [ [0,[-1,-1]],[0,[-1,0]] ], [ [0,[-1,-1]],[0,[-2,0]] ], [ [0,[-1,-1]],[0,[-2,-1]] ] ]
# [ [ [0,[-1,1]],[0,[-2,-1]] ],  [ [0,[-1,1]],[0,[-1,-1]] ],  [ [0,[-1,1]],[0,[0,-1]] ],  [ [0,[-1,1]],[0,[0,0]] ],   [ [0,[-1,1]],[0,[0,1]] ],  [ [0,[-1,1]],[0,[-1,1]] ],  [ [0,[-1,1]],[0,[-2,1]] ],  [ [0,[-1,1]],[0,[-2,0]] ] ]
# [ [ [0,[1,-1]],[0,[0,-2]] ],   [ [0,[1,-1]],[0,[1,-2]] ],   [ [0,[1,-1]],[0,[1,0]] ],   [ [0,[1,-1]],[0,[0,0]] ],   [ [0,[1,-1]],[0,[2,0]] ],  [ [0,[1,-1]],[0,[2,-1]] ],  [ [0,[1,-1]],[0,[2,-2]] ],  [ [0,[1,-1]],[0,[0,-2]] ] ]
# [ [ [0,[1,0]],[0,[2,-1]] ],    [ [0,[1,0]],[0,[1,-1]] ],    [ [0,[1,0]],[0,[0,-1]] ],   [ [0,[1,0]],[0,[0,0]] ],    [ [0,[1,0]],[0,[0,1]] ],   [ [0,[1,0]],[0,[1,1]] ],    [ [0,[1,0]],[0,[2,1]] ],    [ [0,[1,0]],[0,[2,0]] ] ]
# [ [ [0,[0,1]],[0,[-1,0]] ],    [ [0,[0,1]],[0,[0,0]] ],     [ [0,[0,1]],[0,[1,0]] ],    [ [0,[0,1]],[0,[1,1]] ],    [ [0,[0,1]],[0,[1,2]] ],   [ [0,[0,1]],[0,[0,2]] ],    [ [0,[0,1]],[0,[-1,2]] ],   [ [0,[0,1]],[0,[-1,1]] ] ]
# [ [ [0,[-1,0]],[0,[-2,1]] ],   [ [0,[-1,0]],[0,[-1,1]] ],   [ [0,[-1,0]],[0,[0,1]] ],   [ [0,[-1,0]],[0,[0,0]] ],   [ [0,[-1,0]],[0,[0,-1]] ], [ [0,[-1,0]],[0,[-1,-1]] ], [ [0,[-1,0]],[0,[-2,-1]] ], [ [0,[-1,0]],[0,[-2,0]] ] ]
# [ [ [0,[0,-1]],[0,[1,-2]] ],   [ [0,[0,-1]],[0,[0,-2]] ],   [ [0,[0,-1]],[0,[-1,-2]] ], [ [0,[0,-1]],[0,[-1,-1]] ], [ [0,[0,-1]],[0,[-1,0]] ], [ [0,[0,-1]],[0,[0,0]] ],   [ [0,[0,-1]],[0,[1,0]] ],   [ [0,[0,-1]],[0,[1,-1]] ] ]
# [ [ [0,[1,1]],[0,[0,0]] ],     [ [0,[1,1]],[0,[1,0]] ],     [ [0,[1,1]],[0,[2,0]] ],    [ [0,[1,1]],[0,[2,1]] ],    [ [0,[1,1]],[0,[2,2]] ],   [ [0,[1,1]],[0,[1,2]] ],    [ [0,[1,1]],[0,[0,2]] ],    [ [0,[1,1]],[0,[0,1]] ] ]

# Step 5
# put the indidual action-series into a single list


# what is the ideal output?
# the ideal output would come out as a single list such as the following:
# [
#     [ [0,[-1,-1]],[0,[-2,-2]] ], 
#     [ [0,[-1,-1]],[0,[-1,-2]] ], 
#     [ [0,[-1,-1]],[0,[0,-2]] ], 
#     [ [0,[-1,-1]],[0,[0,-1]] ], 
#     [ [0,[-1,-1]],[0,[0,0]] ], 
#     [ [0,[-1,-1]],[0,[-1,0]] ], 
#     [ [0,[-1,-1]],[0,[-2,0]] ], 
#     [ [0,[-1,-1]],[0,[-2,-1]] ]
#
#
# ]

In [ ]:
# What should be happening v2

# Step 1: 
# For each action_series in the full_action_series dictionary, 
# create an equivalent index in the full_action_series dictionary

# full_action_series = {
#         0: [0],
#         1: [1],
#         2: [0,1],
#   }

# full_location_series = {
#         0: [],
#         1: [],
#         2: [],
#   }

# Step 2:





# Final Output
# what the final dictionaries should look like


# full_action_series = {
#         0: [0],
#         1: [1],
#         2: [0,1],
#         ...
#   }


# full_location_series = {
#         0: [ #### full_location_series[index] container ####
#               [-1,-1], # only locations that can be moved to
#               [-1,1],   
#               [1,-1],
#               ...],
#
#         1: [
#               [-1,-1], # only locations where there are enemies
#               [-1,1],
#               [1,-1],
#               ...
#             ],
#
#         2: [
#               [[-1,-1],[-2,-2]], # first the move location, then the target location
#               [[-1,1],[-2,-1]],
#            ],
#   }


In [ ]:
# action_series
# location_series

# each action_series and location_series will need an identifier
# the identifier is for the purpose of identifying which location_series are associated with which action_series in a many (location_series) to one (action_series) relationship

# so what the new process should look like is:
# step 1: create the initial 650 permutations of all the actions
# there will need to be 2 collection lists, we'll call those all_action_series and all_location_series


# step 2: for each action_series, if it contains a 0 or 2 (move or attack), create 8 associated location_series lists
# setting the first number in both as the action_series number, which is the identifier


# step 3: 

In [ ]:
# use a recursive approach to create the process_move_actions function
def generate_move_permutations1(action_series_list, entity, depth=0):
    
    # while this function goes through individual action_series, the function above it (flesh_out_move) goes through a list of action_series
    # action_series_list is supposed to be a single action_series, not a compilation of valid_actions


    if type(action_series_list) == int:
        action_series_list = [action_series_list]
    else:
        action_series_list = list(action_series_list)
    
    #print(f"Action series list: {action_series_list}")

    original_list = deepcopy(action_series_list)
    #print(f"Current depth: {depth}, Current actions: {action_series_list}")
    max_depth = len(action_series_list)
    full_list = []

    if 0 not in action_series_list:
        #print(f"(End 1) Reached end of list. Returning: {action_series_list}")
        #print('')
        full_list.append(action_series_list)
        #print(f"Full list: {full_list}")
        return [action_series_list]

    if depth == 0:
        adjacent_locations_list = adjacent_locations(entity.Location)
        #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

    else:
        # what needs to happen is if it's the first 0 in the list, it needs to check the entity's location
        # first find out if there is another 0
        number_of_zeros = original_list.count(0)
        # then find the number of zeroes before the current depth, they won't be 0s anymore so instead check for lists
        previous_zeros = [item for item in action_series_list if type(item) == list]

        if number_of_zeros == 1:
            adjacent_locations_list = adjacent_locations(entity.Location)
            #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

        elif number_of_zeros > 1:
            if len(previous_zeros) == 0:
                adjacent_locations_list = adjacent_locations(entity.Location)
                #print(f"Adjacent locations (Entity): {adjacent_locations_list}")
            else:
                adjacent_locations_list = adjacent_locations(previous_zeros[-1][1])


    if depth == max_depth:
        #print(f"(End 2) Reached max depth. Returning: {[action_series_list]}")
        #print('')
        full_list.append([action_series_list])
        return [action_series_list]
    
    permutations = []
    for move in adjacent_locations_list: #[[-1,-1], [-1,1], [1,-1], [1,0], [0,1], [-1,0], [0,-1], [1,1]]:

        if action_series_list[depth] == 0:
            #print(f"last move: ",min([abs(depth-1),0]))
            last_move = action_series_list[min([abs(depth-1),0])]
            #print(f"First zero location: {action_series_list[depth]}, Last move: {last_move}")

            new_actions = action_series_list.copy()
            
            #print(f"Current new actions: {new_actions}")
            
            new_actions[depth] = [0, move]

            #print(f"Adding move {move} at depth {depth}")
            full_list.append([new_actions])
            permutations.extend(generate_move_permutations1(new_actions, entity, depth + 1))
            #print('Printing Full List at Add: ', full_list)
            
        else: 
            #print(f"Skipping move {move} at depth {depth}")
            full_list.append(action_series_list)
            permutations.extend(generate_move_permutations1(action_series_list, entity, depth + 1))
            #print('Printing Full List at Skip: ', full_list)

    
    #print(f"Returning permutations at depth {depth}: {permutations}")
    return permutations

def flesh_out_move1(perms, entity):
    fleshed_out_list = []
    perm_result_lengths = []

    num_of_action_series = len(perms)

    #print('Perms: ',perms)


    original_results = []
    for action_series in perms:
        result = generate_move_permutations1(action_series, entity)
        result = list(map(tuple, result))
        

        # I want the final results to be a bit more organized
        action_series_result_length = 0
        for i in result:
            if i not in original_results:
                original_results.extend([i])
                action_series_result_length += 1
        perm_result_lengths.append(action_series_result_length)
        # this counter is to keep track of the number of permutations that have been fleshed out per action series so that the final list can be organized


    #print('Original Results: ',original_results)

    #for i in original_results:
        #if i not in fleshed_out_list:
            #print(f"Point 3a; Adding {i} to fleshed out list")
            #fleshed_out_list.append(original_results)

    #print('Fleshed Out List: ',fleshed_out_list)

    # printing out original results in a more organized way
    for i in range(len(perms)):
        #print(f"Original Action Series: {perms[i]}")

        print('Permutation Result Lengths: ',perm_result_lengths[i])

        action_series_first_result = sum(perm_result_lengths[:i])
        action_series_last_result = sum(perm_result_lengths[:i+1])

        #for j in range(action_series_first_result,action_series_last_result):
            #print(original_results[j])


    #for i in range(num_of_action_series):
    #    print(f"Action Series {i}: {original_results[:sum(perm_result_lengths[:i+1])]}")

    #    for j in range(len(original_results[:sum(perm_result_lengths[:i+1])])):
    #        print(original_results[j])

    print('Point 4; Final Return: ',fleshed_out_list)
    return fleshed_out_list

#flesh_out_move_example_list = [[0,0,0,0], [0,0]]
#result = flesh_out_move1(flesh_out_move_example_list, actor)


In [ ]:
def generate_turns():
    all_action_series = []

    for length in range(1, 9):  # Turns can be 1 to 8 actions long: 6 Moves, 1 Object Interaction, 1 Action
        for turn in itertools.product(range(6), repeat=length):
            if validate_turn(turn):
                yield turn
                
    return all_action_series


def validate_turn(turn):
    move_count = sum(1 for action in turn if action == 0)
    object_interact_count = sum(1 for action in turn if action in [1, 3])
    action_count = sum(1 for action in turn if action in [2, 3, 4, 5, 6])
        
    return (move_count <= 6 and 
            object_interact_count <= (2 if 6 in turn else 1) and 
            action_count <= 1)


def calc_rewards(turn, actor):

    turn_conciseness = 0

    turn_evaluation = 0

    reward = 0
    turn_evaluation += reward
    return turn_evaluation


In [ ]:
def generate_move_permutations2(action_series_dict, location_series_dict, key, world, entity, depth=0):
    action_series = action_series_dict[key]
    location_series_container = location_series_dict[key]

    action_series = list(action_series)
    max_depth = len(action_series)

    location_series_placeholder = action_series.copy()


    if depth == 0:
        adjacent_locations_list = adjacent_locations(entity.Location) 
    else:
        number_of_zeroes = action_series.count(0)
        previous_zeroes = [item for item in location_series_placeholder if type(item) == list]

        if number_of_zeroes == 1:
            adjacent_locations_list = adjacent_locations(entity.Location)
        elif number_of_zeroes > 1:
            if len(previous_zeroes) == 0:
                adjacent_locations_list = adjacent_locations(entity.Location)
            else:
                adjacent_locations_list = adjacent_locations(previous_zeroes[-1])

    if depth == max_depth:
        location_series_dict[key].append(action_series)
        return location_series_dict

    for move in adjacent_locations_list:
        if action_series[depth] == 0:
            new_actions = action_series.copy()
            new_actions[depth] = [0, move]
            location_series_dict[key].append(new_actions)
            generate_move_permutations2(action_series_dict, location_series_dict, key, world, entity, depth + 1)
        else:
            location_series_dict[key].append(action_series)
            generate_move_permutations2(action_series_dict, location_series_dict, key, world, entity, depth + 1)
    
    return location_series_dict

    

def flesh_out_move2(action_series_dict, location_series_dict, entity):
    for key in range(len(action_series_dict)):
        print(key)
        if 0 in action_series_dict[key]:
            generate_move_permutations2(action_series_dict, location_series_dict, key, stage, entity)
    return location_series_dict
    
    # what is the purpose of the flesh_out_move function compared to the generate_move_permutations function?
    # so if flesh_out_move is the function meant to fully process the 0s in the action options, 
    # and it iterates through the key values of the action_series_dict,
    # then generate_move_permutations only needs to 

In [ ]:
# here's where the Attack action will be fleshed out
def generate_attack_permutations(action_series, world, entity, depth=0):
    pass

    # the way the attack permutations will work is based from the world.Enemy_Locations list
    
    # should melee vs ranged weapons be considered?

    # lets only consider melee weapons for now

    # the attack action will only be valid if the target is within 1 space of the entity
    # because the entity must be within 1 space of the target to attack, 
    # the last location assigned to a move action in the action_series_list can be used as a reference for the entity's location

def flesh_out_attack(action_series_dict, location_series_dict, entity):
    pass 

In [ ]:
def remove_redundant_series(perms):
    pass

# what would be considered redundant?
# - different movement patterns through terrain with no difference in properties


In [ ]:
# need to do both the move and attack actions while removing redundant permutations in the same function
def generate_move_and_attack_permutations(key, action_series_dict, location_series_dict, world, entity, depth=0):

    action_series = action_series_dict[key]
        
    if type(action_series) != list:
        action_series = list(action_series)

    print(f"Action series list: {action_series}")

    final_list_to_be_assigned_to_location_series_dict = []

    ###### ---------------------- Ending the Recursion --------------------------- ######
    max_depth = len(action_series)
    if depth == max_depth:
        print(f"(End 2) Reached max depth. Returning: {action_series}")
        #location_series_dict[key].append(action_series) #commenting this out removed the issue of a 0 appearing where it shouldn't
        print('Location_series_dict[key]: ', location_series_dict[key])
        print('')
        return


    location_series = action_series.copy()
    # the location series is to be the...
    #   a) the list of lists that gets assigned to the location_series_dict[key]
    #   b) the individual list that within gets assigned the iterating adjacent_locations
    
    # I need to make it clear in my head what the difference between location_series and new_action_series_per_location is
    #  i)  location_series - 
    #  ii) new_action_series_per_location -

    global new_action_series_per_location
    new_action_series_per_location = action_series.copy()
    

    ###### ------------------ Creating the Adjacent Location List ----------------- ######
    print('')
    print('Creating Adjacent Location List')
    print(f"Current depth: {depth}, Current actions: {action_series}")

    

    if depth == 0:
        dedicated_location = entity.Location
        location_series = action_series.copy()
        adjacent_locations_list = adjacent_locations(dedicated_location)
        #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

    else:
        number_of_zeroes = action_series.count(0)
        previous_zeroes = [item for item in range(len(location_series[:depth])) if type(item) == list]
        print(f"Number of Zeroes: {number_of_zeroes}")
        print(f"Location Series Placeholder: {location_series}")
        print(f"Previous Zeroes: {previous_zeroes}")

        if number_of_zeroes == 1:
            dedicated_location = entity.Location
            adjacent_locations_list = adjacent_locations(dedicated_location)
            #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

        elif number_of_zeroes > 1:
            if len(previous_zeroes) == 0:
                dedicated_location = entity.Location
                adjacent_locations_list = adjacent_locations(dedicated_location)
                #print(f"Adjacent locations (Entity): {adjacent_locations_list}")

            else:
                dedicated_location = previous_zeroes[-1][1]
                adjacent_locations_list = adjacent_locations(dedicated_location)
                #print(f"Adjacent locations (Space): {adjacent_locations_list}")

    print(f'Created Locations Adjacent with {dedicated_location}')



    ###### Generating Permutations ######
    print('')
    print('Generating Permutations')

    for space in adjacent_locations_list:

        if action_series[depth] == 0:
            # if new_action_series_per_location already has a list in it, I don't want to rewrite it
            #if list not in new_action_series_per_location:
            #    new_action_series_per_location = action_series.copy()

            
            print(f"Adding move {space} at depth {depth} in {new_action_series_per_location}")
            new_action_series_per_location[depth] = [space]
            print("New Action Series at Location (0): ", new_action_series_per_location)

            location_series.append([new_action_series_per_location]) # I just replaced location_series_dict[key] with location_series
            print('Location Series: ', location_series)
            generate_move_and_attack_permutations(key, action_series_dict, location_series_dict, world, entity, depth + 1)
        
        elif action_series[depth] == 2:
            print('Attack Action')
            adjacent_enemies = [enemy for enemy in world.Enemy_Locations if chebyshev_distance(space, enemy) == 1]
            if len(adjacent_enemies) == 0:
                continue
            else:
                print(f"Skipping move {space} at depth {depth}")

                new_action_series_per_location = action_series.copy()
                new_action_series_per_location[depth] = [space]
                print("New Action Series at Location (2): ", new_action_series_per_location)


                location_series_dict[key].append(new_action_series_per_location)
                generate_move_and_attack_permutations(key, action_series_dict, location_series_dict, world, entity, depth + 1)
    
    #location_series_dict[key].append(action_series)
    #print(f"Returning permutations at depth {depth}: {action_series}")




    # in which order should the move and attack actions be checked?



    # which moves to remove and how to remove them
    # a move should be removed if there isn't an enemy within range to attack
    # for enemy in world.Enemy_Locations:
    #    if chebyshev_distance(entity.Location, enemy) > 1:
    #       return




def flesh_out_actions(action_series_dict, location_series_dict, world, entity):
    for key in range(len(action_series_dict)):
        print('')
        print(f"Key: {key}")
        print('')
        if 0 in action_series_dict[key] or 2 in action_series_dict[key]:
            generate_move_and_attack_permutations(key, action_series_dict, location_series_dict, world, entity)
    return location_series_dict


In [ ]:
# What should be happening v2

# Step 1: 
# For each action_series in the full_action_series dictionary, 
# create an equivalent index in the full_action_series dictionary

# full_action_series = {
#         0: [0],
#         1: [1],
#         2: [0,1],
#   }

# full_location_series = {
#         0: [],
#         1: [],
#         2: [],
#   }

# Step 2:





# Final Output
# what the final dictionaries should look like


# full_action_series = {
#         0: [0],
#         1: [1],
#         2: [0,1],
#         ...
#   }


# full_location_series = {
#         0: [ #### full_location_series[index] container ####
#               [-1,-1], # only locations that can be moved to
#               [-1,1],   
#               [1,-1],
#               ...],
#
#         1: [
#               [-1,-1], # only locations where there are enemies
#               [-1,1],
#               [1,-1],
#               ...
#             ],
#
#         2: [
#               [[-1,-1],[-2,-2]], # first the move location, then the target location
#               [[-1,1],[-2,-1]],
#            ],
#   }


In [ ]:
# given an input of an action_series from action_series_dict[key],
# the ideal output is a list of lists, 
# where the individual lists are all possible locations for each location based action in the action_series

# so if we're given the action_series, 
# there are a few points that create branches

In [ ]:
def adjacent_locations(pos):
    adjacent_locations_list = []
    # for an 8 directional grid
    for i in range(-1,2):
        for j in range(-1,2):
            adjacent_locations_list.append([pos[0]+i,pos[1]+j])
    adjacent_locations_list.remove(pos)
    return adjacent_locations_list



In [3]:
from itertools import chain, combinations

# Chebyshev 
def chebyshev_distance(pos1,pos2):
    return max(abs(pos1[0]-pos2[0]),abs(pos1[1]-pos2[1]))

# create a power set of all coordinates 6 spaces away from (0,0) according to the Chebyshev distance
locations = []
for x in range(-6,7):
    for y in range(-6,7):
        if chebyshev_distance((0,0),[x,y]) <= 6:
            locations.append((x,y))
print(locations)
print(len(locations))


# I should order the locations by the Chebyshev distance from the origin
locations = sorted(locations,key=lambda x: chebyshev_distance((0,0),x))



def is_legal(sequence, origin=(0,0)):
    # is the first entity the origin, if not false
    if sequence == []:
        return False
    elif sequence[0] != origin:
        return False

    # is the length 8 or less
    if len(sequence) > 8:
        return False
    
    # is the distance between each entity 1 or less
    for i in range(len(sequence)-1):
        if chebyshev_distance(sequence[i],sequence[i+1]) > 1:
            return False
    
    return True


def classical_recursive_one(elems):
    yield [] # first return the result we’re sure about 
    for i in range(len(elems)):
        for x in classical_recursive_one(elems[i+1:]): 
            # induction part
            yield [elems[i]] + x
             

def classical_iterative(elems):
    powerset_size = 2**len(elems)
    print('Powerset Size: ',powerset_size)
    counter = 0
    j = 0
 
    for counter in range(0, powerset_size):
        print(counter)
        results = []
        for j in range(0, len(elems)):
            # take the element if on bit position j it says to take it (i.e. 1 appears)
            if((counter & (1 << j)) > 0):
                results.append(elems[j])
        if is_legal(results):
            yield results
        else:
            continue

def powerset(iterable):
    "powerset([1,2,3]) → () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    results = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    
    good_results = []
    for result in results:
        if is_legal(list(result)):
            good_results.append(result)
            yield result
        else:
            continue

    print('Good Results: ',len(good_results))
    return good_results

powerset_results = powerset(locations)
powerset_results

#for x in classical_recursive_one(locations):
#for x in classical_iterative(locations):

#for x in powerset_results:
#    print(x)

#moves = classical_iterative(locations)


[(-6, -6), (-6, -5), (-6, -4), (-6, -3), (-6, -2), (-6, -1), (-6, 0), (-6, 1), (-6, 2), (-6, 3), (-6, 4), (-6, 5), (-6, 6), (-5, -6), (-5, -5), (-5, -4), (-5, -3), (-5, -2), (-5, -1), (-5, 0), (-5, 1), (-5, 2), (-5, 3), (-5, 4), (-5, 5), (-5, 6), (-4, -6), (-4, -5), (-4, -4), (-4, -3), (-4, -2), (-4, -1), (-4, 0), (-4, 1), (-4, 2), (-4, 3), (-4, 4), (-4, 5), (-4, 6), (-3, -6), (-3, -5), (-3, -4), (-3, -3), (-3, -2), (-3, -1), (-3, 0), (-3, 1), (-3, 2), (-3, 3), (-3, 4), (-3, 5), (-3, 6), (-2, -6), (-2, -5), (-2, -4), (-2, -3), (-2, -2), (-2, -1), (-2, 0), (-2, 1), (-2, 2), (-2, 3), (-2, 4), (-2, 5), (-2, 6), (-1, -6), (-1, -5), (-1, -4), (-1, -3), (-1, -2), (-1, -1), (-1, 0), (-1, 1), (-1, 2), (-1, 3), (-1, 4), (-1, 5), (-1, 6), (0, -6), (0, -5), (0, -4), (0, -3), (0, -2), (0, -1), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, -6), (1, -5), (1, -4), (1, -3), (1, -2), (1, -1), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, -6), (2, -5), (2, -4), (2, -3), (2, -

<generator object powerset at 0x0000018B5E53B890>

In [ ]:
# lets create the power set for action_series
from itertools import chain, combinations

subaction_dict = {
    '0': 'Move',                # needs a target
    '1': 'Object (Bonus)',      # needs a target
    '2': 'Attack',              # needs a target
    '3': 'Dash',
    '4': 'Disengage',
    '5': 'Dodge',
    '6': 'Object',              # needs a target
}



def generate_turns():
    all_action_series = []

    for length in range(1, 9):  # Turns can be 1 to 8 actions long: 6 Moves, 1 Object Interaction, 1 Action
        for turn in itertools.product(range(6), repeat=length):
            if validate_turn(turn):
                yield turn
    
    return all_action_series

def validate_turn(turn):
    move_count = sum(1 for action in turn if action == 0)
    object_interact_count = sum(1 for action in turn if action in [1, 3])
    action_count = sum(1 for action in turn if action in [2, 3, 4, 5, 6])
        
    return (move_count <= 6 and 
            object_interact_count <= (2 if 6 in turn else 1) and 
            action_count <= 1)

def choose_turn():
    turn_options = generate_turns()
    all_action_series = {i:turn for i,turn in enumerate(turn_options) if validate_turn(turn)}
    return all_action_series

choose_turn()

In [ ]:
from collections import deque
import igraph
from igraph import Graph, EdgeSeq
import plotly.graph_objects as go


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

def create_tree_visualization_with_integer_coordinates(nr_vertices=10, children=8):
    # Create a tree using NetworkX
    depth = int(np.log(nr_vertices) / np.log(children))
    G = nx.balanced_tree(children, depth)
    
    # Create a custom layout with integer coordinates
    pos = {}
    max_width = children ** depth
    
    def assign_positions(node, x, y, width):
        pos[node] = (x, -y)  # Negative y to have root at the top
        if G.degree(node) > 1 or (node == 0 and G.degree(node) > 0):  # Check for children
            new_width = width // children
            child_nodes = list(G.neighbors(node))
            for i, child in enumerate(child_nodes):
                new_x = x - width // 2 + new_width // 2 + i * new_width
                assign_positions(child, new_x, y + 1, new_width)
    
    assign_positions(0, max_width // 2, 0, max_width)
    
    # Prepare edge coordinates
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    # Create the plot
    plt.figure(figsize=(20, 15))
    
    # Plot edges
    plt.plot(edge_x, edge_y, 'gray', alpha=0.5)
    
    # Plot nodes
    x_values = [pos[k][0] for k in pos]
    y_values = [pos[k][1] for k in pos]
    plt.scatter(x_values, y_values, s=100, c='lightblue', edgecolors='black', zorder=2)
    
    # Add labels with integer coordinates
    for node, (x, y) in pos.items():
        label = f"[{x}, {-y}]"  # Negative y to display positive y-coordinates
        plt.text(x, y, label, ha='center', va='center', fontweight='bold', fontsize=8)
    
    plt.title(f"Tree with {len(G.nodes())} vertices and {children} children per node")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Call the function
create_tree_visualization_with_integer_coordinates(nr_vertices=10, children=8)

In [ ]:
nr_vertices = 100
v_label = list(map(str, range(nr_vertices)))
G = Graph.Tree(nr_vertices, 8) # 2 stands for children number
lay = G.layout('rt')

position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

labels = v_label

def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k], # or replace labels with a different list for the text within the circle
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    return annotations

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(
              annotations=make_annotations(position, v_label),
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              hovermode='closest',
              )
fig.show()

In [ ]:
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)


def depth_first_traversal(node):
    print(node.value)
    for child in node.children:
        depth_first_traversal(child)


def level_order_traversal(root):
    queue = deque([root])
    
    while queue:
        current_node = queue.popleft()
        print(current_node.value)
        queue.extend(current_node.children)

In [ ]:
root = TreeNode("Beverages")

hot = TreeNode("Hot")
cold = TreeNode("Cold")

root.add_child(hot)
root.add_child(cold)

In [ ]:
print("Depth-First Traversal:")
depth_first_traversal(root)

In [ ]:
print("Level-Order Traversal:")
level_order_traversal(root)

In [ ]:
def search(value, node):
    if node.value == value:
        return node
    for child in node.children:
        result = search(value, child)
        if result:
            return result
    return None

In [ ]:
soda_node = search("Soda", root)
if soda_node:
    print(f"Found node: {soda_node.value}")

In [ ]:
def generate_move_and_attack_permutations2():
    pass

def flesh_out_actions2():
    pass

In [ ]:
def Choose_Turn(world, entity, show_turns: bool):
    turn_options = generate_turns()
    all_action_series = {i:turn for i,turn in enumerate(turn_options) if validate_turn(turn)}
    all_location_series = {key:[] for key in all_action_series.keys()}

    # I only want the first 100 key value pairs
    all_action_series = {key:all_action_series[key] for key in range(100)}
    all_location_series = {key:all_location_series[key] for key in range(100)}

    print('Before Fleshing Out:',len(all_action_series))

    # the next thing that happens is the move and attack actions are fleshed out
    #flesh_out_move2(all_action_series, all_location_series, entity)

    flesh_out_actions2(all_action_series, all_location_series, world, entity)

    print(all_action_series)
    print(all_location_series)
    # only do the first 100
    # fleshed_out_movement_turns = []
    # for turn in valid_turns:
        #print(turn)
    #     new_turn = flesh_out_move(turn,entity)
    #    print(new_turn)
    #    fleshed_out_movement_turns.extend(new_turn)


    # if show_turns:
    #    print('Valid Turns:', len(fleshed_out_movement_turns))
    #    for turn in fleshed_out_movement_turns:
    #        print(turn)
    
    #best_turn = None
    #best_score = float('-inf')

    #for turn in fleshed_out_movement_turns:
    #    score = calc_rewards(turn, entity)
    #    if score > best_score:
    #        best_turn = turn
    #        best_score = score
    #        print(best_score)

    #best_turn = list(best_turn)
    #return best_turn



In [ ]:
#Choose_Turn(stage, actor, False)

# takes 20 minutes to process 261/650